# Toy Problem Demo

## Scope

Overview of Benchq:
- inputs and outputs
- components:
  - Transpilation (pyliqtr)
  - Jabalizer/ICM
  - Min code distance finding
  - Substrate scheduling


Wait a little for TA 1 and TA 1.5

### Goal: Introduction to Benchq Tool for Resource Estimation

## Agenda
- Quick overview of Benchq usage.
- What is a circuit graph? How do we produce them?
- How do circuit graphs help get resource estimates?
- Look at some pretty plots
  - Preparing a GHZ state
  - The fully connected graph

## Preliminaries

First we need to import some things and install a python environment in your python installation. This might take a minute.

In [7]:
%matplotlib inline
from qiskit.circuit import QuantumCircuit
import os
import json
import networkx as nx

from benchq import BasicArchitectureModel
from benchq.compilation import get_algorithmic_graph_from_Jabalizer, pyliqtr_transpile_to_clifford_t
from benchq.resource_estimation.graph import (
    graph_estimator,
)

# Running this line will install julia to your python installation. See README for details.

In [2]:
from benchq.algorithms.data_structures import BASIC_SC_ARCHITECTURE_MODEL

demo_circuit = QuantumCircuit.from_qasm_file("data/single_rotation.qasm")
architecture_model = BASIC_SC_ARCHITECTURE_MODEL

Next, transpile into Clifford + T.

In [ ]:
clifford_t_circuit = pyliqtr_transpile_to_clifford_t(
    demo_circuit, circuit_precision=1e-6
)
print(clifford_t_circuit)

Transform circuit into graph.

In [4]:
circuit_graph = get_algorithmic_graph_from_Jabalizer(clifford_t_circuit)

With this use this graph to make resource estimates.

In [ ]:
from benchq.resource_estimation.graph import GraphResourceEstimator
from benchq.algorithms.data_structures import (
    AlgorithmImplementation,
    ErrorBudget,
    GraphPartition,
    QuantumProgram,
)

budget = ErrorBudget.from_even_split(
    1e-2
)  # only allow a failure to occur 1% of the time
program = GraphPartition(
    QuantumProgram.from_circuit(clifford_t_circuit), [circuit_graph]
)
implementation = AlgorithmImplementation(program, budget, 1)
estimator = GraphResourceEstimator(architecture_model)

resource_estimates = estimator.estimate(implementation)
print(resource_estimates)

### Summary

#### Minimal Inputs:
- Circuit
- Archetecture Model

#### Outputs:
- Number of physical qubits
- Computation time
- number of measurement steps (will be important later on!)

### Pipelines

#### Example usage

Typically, one would use a pipeline to combine all these steps into one command that benchq can follow.

Here we use one of the pre-configured pipelines called `run_precise_graph_estimate`.

In [ ]:
from benchq.resource_estimation.default_pipelines import (
    run_precise_graph_estimate,
)
from benchq.conversions import import_circuit

budget = ErrorBudget.from_even_split(
    1e-2
)  # only allow a failure to occur 1% of the time
program = QuantumProgram.from_circuit(import_circuit(demo_circuit))

implementation = AlgorithmImplementation(program, budget, 1)

resource_estimates = run_precise_graph_estimate(implementation, architecture_model)
print(resource_estimates)

#### Default Pipelines
Benchq offers 4 default pipelines:

1. `run_precise_graph_estimate` - This pipeline uses the most accurate methods for estimating the resources required to run a circuit on a given architecture. It is also the slowest.
2. `run_fast_graph_estimate` - This pipeline uses faster methods for estimating the resources required to run a circuit on a given architecture. It is also less accurate.
3. `run_precise_extrapolation_estimate` - This pipeline uses the most accurate methods for estimating the resources required to get part of the graph. It will then extrapolate out to the size of the full graph.
4. `run_fast_extrapolation_estimate` - This pipeline uses the fastest methods for estimating the resources required to get part of the graph. It will then extrapolate out to the size of the full graph.

They can be imported with the following command:

```python
import benchq.resource_estimation.graph import <pipeline name>
```

And run with the same inputs as above.

## What is a a Circuit Graph?

In [7]:
this_circuit_graph = get_algorithmic_graph_from_Jabalizer(clifford_t_circuit)

### What does this do?

Recall that our circuit is in clifford + T form

- Replaces T gates with magic measurements and ancilla
- Use stabilizer simulator efficiently to push single qubit cliffords to one side
- Now we have a circuit of the form Initialization, CNOT, Measurement (ICM form)

In [10]:
print(clifford_t_circuit)

Circuit(operations=[H(0), S†(0), H(0), S(0), T(0), H(0), S(0), T(0), H(0), T(0), H(0), T(0), H(0), S(0), T(0), H(0), T(0), H(0), T(0), H(0), T(0), H(0), S(0), T(0), H(0), S(0), T(0), H(0), T(0), H(0), T(0), H(0), S(0), T(0), H(0), T(0), H(0), T(0), H(0), T(0), H(0), T(0), H(0), S(0), T(0), H(0), T(0), H(0), S(0), T(0), H(0), S(0), T(0), H(0), T(0), H(0), S(0), T(0), H(0), T(0), H(0), T(0), H(0), T(0), H(0), T(0), H(0), T(0), H(0), T(0), H(0), S(0), T(0), H(0), S(0), T(0), H(0), S(0), T(0), H(0), S(0), T(0), H(0), T(0), H(0), T(0), H(0), T(0), H(0), S(0), T(0), H(0), S(0), T(0), H(0), T(0), H(0), T(0), H(0), T(0), H(0), S(0), T(0), H(0), S(0), T(0), H(0), T(0), H(0), S(0), T(0), H(0), S(0), T(0), H(0), T(0), H(0), S(0), T(0), H(0), S(0), T(0), H(0), S(0), T(0), H(0), T(0), H(0), S(0), T(0), H(0), T(0), H(0), T(0), H(0), T(0), H(0), S(0), T(0), H(0), S(0), T(0), H(0), S(0), T(0), H(0), S(0), T(0), H(0), T(0), H(0), T(0), H(0), S(0), T(0)], n_qubits=4)


In [11]:
import data.get_icm as icm

circuit_after_icm = icm.get_icm(clifford_t_circuit)
print(circuit_after_icm)

Circuit(operations=[H(0), S†(0), H(0), S(0), CNOT(0,4), H(4), S(4), CNOT(4,5), H(5), CNOT(5,6), H(6), CNOT(6,7), H(7), S(7), CNOT(7,8), H(8), CNOT(8,9), H(9), CNOT(9,10), H(10), CNOT(10,11), H(11), S(11), CNOT(11,12), H(12), S(12), CNOT(12,13), H(13), CNOT(13,14), H(14), CNOT(14,15), H(15), S(15), CNOT(15,16), H(16), CNOT(16,17), H(17), CNOT(17,18), H(18), CNOT(18,19), H(19), CNOT(19,20), H(20), S(20), CNOT(20,21), H(21), CNOT(21,22), H(22), S(22), CNOT(22,23), H(23), S(23), CNOT(23,24), H(24), CNOT(24,25), H(25), S(25), CNOT(25,26), H(26), CNOT(26,27), H(27), CNOT(27,28), H(28), CNOT(28,29), H(29), CNOT(29,30), H(30), CNOT(30,31), H(31), CNOT(31,32), H(32), S(32), CNOT(32,33), H(33), S(33), CNOT(33,34), H(34), S(34), CNOT(34,35), H(35), S(35), CNOT(35,36), H(36), CNOT(36,37), H(37), CNOT(37,38), H(38), CNOT(38,39), H(39), S(39), CNOT(39,40), H(40), S(40), CNOT(40,41), H(41), CNOT(41,42), H(42), CNOT(42,43), H(43), CNOT(43,44), H(44), S(44), CNOT(44,45), H(45), S(45), CNOT(45,46), H(46

The middle CNOTS are the interesting part:

- The CNOTS make a graph state
- Use stabilizer simulator to find graph state (Jabalizer)
- Return graph state as circuit graph

In [12]:
nx.draw(circuit_graph, node_size=10)

Here we see a simple chain graph because our circuit structure simpy passes the qubits down the line generated by decomposing to clifford + T.

### Summary

Circuit graphs are generated from the icm form of a circuit.

They are a representation of the graph state that is generated by the circuit.

## Getting Resource Estimates from Circuit Graphs

Rather than keep the whole graph, we extract the information we need to get resource estimates.

We store this in a `GraphData` object.

Let's make a `GraphData` object from our demo circuit.

In [13]:
from benchq.algorithms.data_structures import GraphPartition

# we have to wrap `circuit_graph` in a `GraphPartition` object,
# but you can ignore that as it is usually done internally
graph_partition = GraphPartition(program, [circuit_graph])

graph_data = estimator._get_graph_data_for_single_graph(graph_partition)
print(graph_data)

getting max graph degree
starting substrate scheduler
substrate scheduler took 0.0002589225769042969 seconds
GraphData(max_graph_degree=2, n_nodes=63, n_t_gates=0, n_rotation_gates=1, n_measurement_steps=3)


As you can see, the graph data involves some data that comes from how the graph was made including `n_t_gates` and `n_rotation_gates`.

`n_measurement_steps` is the number of measurement steps required to prepare that graph_state. It is used to calculate the time required to prepare the graph state.

Next, let's use `graph_data`` to get some resource estimates.

In [ ]:
from benchq.magic_state_distillation import MagicStateFactory

# one of Litinski's magic state distillation factories
magic_state_factory = MagicStateFactory("(15-to-1)_11,5,5", 1.9e-11, (47, 44), 2070, 30)

logical_qubit_count = len(circuit_graph)

n_total_t_gates = estimator.get_n_total_t_gates(
    graph_data.n_t_gates,
    graph_data.n_rotation_gates,
    budget.transpilation_failure_tolerance,
)

distance = estimator._minimize_code_distance(
    n_total_t_gates,
    graph_data,
    architecture_model.physical_qubit_error_rate,  # physical error
    magic_state_factory,
)
physical_qubit_count = estimator._get_n_physical_qubits(graph_data, distance)
total_time = estimator._get_time_per_circuit_in_seconds(
    graph_data, distance, n_total_t_gates
)


print(f"distance: {distance}")
print(f"physical qubit count: {physical_qubit_count}")
print(f"total time: {total_time}")

And there you go! We have some resource estimates.

Don't worry  if the estimates are a little high, that's just because we didn't choose a smaller magic state factory for this calculation.

### How to make Circuit Graph State?

Since graph state is a stabilizer state, we measure stabilizers to generate it!

We could measure all the stabilizers to get the graph.

Measurements are expensive!! So how optimize?

### Substrate Scheduler

Tells us how to measure and which can be measured simultaneously.

In [27]:
from benchq.resource_estimation.graph.graph_estimator import substrate_scheduler

compiler = substrate_scheduler(circuit_graph, "fast")
formatted_measurement_steps = [
    [node[0] for node in step] for step in compiler.measurement_steps
]

print(formatted_measurement_steps)

[[0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60], [1, 4, 7, 10, 13, 16, 19, 22, 25, 28, 31, 34, 37, 40, 43, 46, 49, 52, 55, 58, 61], [2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38, 41, 44, 47, 50, 53, 56, 59, 62]]


In [28]:
from benchq.vizualization_tools import plot_graph_state_with_measurement_steps

plot_graph_state_with_measurement_steps(
    compiler.input_graph, compiler.measurement_steps
)

### Problem! Graph can get too big to handle!

#### Solution! Use subcircuits.

Quantum Algorithms are made up of repeated components.

Estimate resources for each component & multiply by the number of times it was used.

Will create a higher estimate.

More on this later!

## FINALLY! Pretty Graph Time!

Let's look at the graphs of circuits to examine measurement steps!

In [34]:
circuit = QuantumCircuit.from_qasm_file("data/ghz_circuit.qasm")

clifford_t_circuit = pyliqtr_transpile_to_clifford_t(circuit, circuit_precision=1e-10)
circuit_graph = get_algorithmic_graph_from_Jabalizer(clifford_t_circuit)
compiler = substrate_scheduler(circuit_graph, "fast")

plot_graph_state_with_measurement_steps(
    compiler.input_graph, compiler.measurement_steps
)

In [35]:
circuit = QuantumCircuit.from_qasm_file("data/h_chain_circuit.qasm")


clifford_t_circuit = pyliqtr_transpile_to_clifford_t(circuit, circuit_precision=1e-10)
circuit_graph = get_algorithmic_graph_from_Jabalizer(clifford_t_circuit)
compiler = substrate_scheduler(circuit_graph, "fast")

plot_graph_state_with_measurement_steps(
    compiler.input_graph, compiler.measurement_steps
)

/Users/athenacaesura/.pyenv/versions/3.10.11/envs/fix-toy-problem/lib/python3.10/site-packages/pyLIQTR/gate_decomp/cirq_transforms.py:519: RuntimeWarning: CNOT is not a rotation gate, cannot decompose. Set warn_if_not_decomposed=False to silence this warning.
  warnings.warn(
/Users/athenacaesura/.pyenv/versions/3.10.11/envs/fix-toy-problem/lib/python3.10/site-packages/pyLIQTR/gate_decomp/cirq_transforms.py:519: RuntimeWarning: H is not a rotation gate, cannot decompose. Set warn_if_not_decomposed=False to silence this warning.
  warnings.warn(


## Closing Statements

### What did we learn?


#### Inputs
- Circuit
- Architecture model
#### Outputs
- Number of physical qubits
- Computation time
- Number of measurement steps



#### Components:
- Transpilation (pyliqtr)
  - Bring to Clifford + T
- Jabalizer/ICM
  - Easy way to represent circuit
- Min code distance finding
  - Number of physical qubits
  - Computation time
- Substrate scheduling
  - number of measurement steps

## What's Next?

- How to get resource estimate for large algorithms? (QuantumPrograms)
- Compare to other resource estimators.
- Try this notebook out for yourself!!